# Project 3
## Author: Brailey Sharpe
## Version: Fall 2025

In [4]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv('data/COMP3800F25S01RawData.csv', low_memory=False)
print(df.shape)

(21534, 1891)
